<a href="https://colab.research.google.com/github/TheGreatestEvers/ADL4CV/blob/dev_evaluation/zero-shot/test_evaluation_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Clone Private GitHub Repository
import os
from getpass import getpass

# Replace with the repository URL
repo_url = "https://github.com/TheGreatestEvers/ADL4CV.git"

# Enter your GitHub username
username = "maxdoesch"

# Enter your personal access token
token = getpass('Enter your GitHub personal access token: ')

# Construct the full repository URL with the token
repo_url_with_token = repo_url.replace("https://", f"https://{username}:{token}@")

# Clone the repository
!git clone -b dev_evaluation {repo_url_with_token}


In [ ]:
%cd ADL4CV
!pip install -r requirements.txt

In [ ]:
!wget https://storage.googleapis.com/dm-tapnet/tapvid_davis.zip
!unzip tapvid_davis.zip
%cd zero-shot

In [ ]:
import torch
import numpy as np
from PIL import Image, ImageSequence

from diffusion_wrapper import DiffusionWrapper
from heatmap_generator import HeatmapGenerator
from zero_shot_tracker import ZeroShotTracker


diffusion_wrapper = DiffusionWrapper()
heatmap_generator = HeatmapGenerator()
zero_shot_tracker = ZeroShotTracker()

In [ ]:
from evaluation_datasets import create_davis_dataset

davis_dataset = create_davis_dataset('../tapvid_davis/tapvid_davis.pkl')

first_video = {}

for data in davis_dataset:
    first_video = data['davis']
    break

print(first_video['video'].dtype)

video_tensor = torch.tensor(first_video['video'])
print(video_tensor.shape)

In [ ]:
video_features_dict = diffusion_wrapper.extract_video_features(video_tensor, "")

In [ ]:
from algorithms.feature_extraction_loading import concatenate_video_features

video_features = concatenate_video_features({'up_block': video_features_dict['up_block'][1:3]})
print(video_features.shape)

In [ ]:
video_features = video_features.permute(0, 2, 3, 1).float()
heatmaps = heatmap_generator.generate(video_features, (93, 137, 0))
tracks = zero_shot_tracker.track(heatmaps)

In [ ]:
zero_shot_tracker.place_marker_in_frames(first_video['video'].squeeze(), tracks)
heatmap_generator.safe_heatmap_as_gif(heatmaps)